# 5315 - PEST Modelo de Coposa

A través de este Jupyter se armará la interfaz de PEST para calibrar el modelo de Coposa:

- Generación de archivos de niveles y sulfato para usar en la calibración.
- Generación de archivos de caudales para usar en la calibración.



In [1]:
%matplotlib inline
import os
import shutil
import platform
import numpy as np
import pandas as pd
from matplotlib.patches import Rectangle as rect
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", 
    message="ModflowDis.sr is deprecated. use Modflow.sr")
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
newparams = {'legend.fontsize':10, 'axes.labelsize':10,
             'xtick.labelsize':10, 'ytick.labelsize':10,
             'font.family':'Univers 57 Condensed', 
             'pdf.fonttype':42}
plt.rcParams.update(newparams)
import pyemu

from TargetsModule import *
from PEST_ACL import *
from readAnyCsv import *

import flopy

flopy is installed in C:\ProgramData\Anaconda3\lib\site-packages\flopy


# Generales

Algunas definiciones generales que se usarán a lo largo del notebook.

In [4]:
# Ruta donde se realizará el pesteo
ruta_pesteo='50 Pesteo\\'
# Root file name
pack_name='5315_Calibracion'
# Periodos del modelo
sp_totales=277
# Nodos del modelo
number_of_nodes = 199395
# Fecha y hora inicial del modelo
fecha_ini='30/12/1997'
hora_ini='00:00:00'

# Targets

En esta sección se busca crear todo lo relacionado a los targets de niveles (y/o sulfato) que necesita PEST para calibrar. Se crean usando la función heads2all del módulo TargetsModule que recibe como inputs los mismos archivos con que se incorporan targets a GWV (info y serie transiente), el root file name de los paquetes, una fecha y hora inicial, y la carpeta donde se realizará el proceso de calibración.

Finalemte los archivos que se generan son:

- Archivo de niveles observados (niveles.smp) que toma el usgmod2obs.
- Archivos necesarios para situar los niveles en la grilla (coord.dat y interp.dat).
- Instrucción del programa usgmod2obs para generar los niveles simulados en el modelo.
- Instrucción de PEST para extraer los niveles simualdos generados a través de usgmod2obs. 

In [3]:
ruta_targets='01 Targets\\'

# Archivos que conforman los targets. Mismos que a GWV.
info=pd.read_csv(ruta_targets+'00_info_niveles.csv',',')
transiente=pd.read_csv(ruta_targets+'01_serie_niveles.csv',',')

prefijo_heads='h'
mod2obs_in='to_mod2obs.in'
sim_smp_file='niveles_sim.smp'
pest_ins='niveles_sim.ins'

heads2all(info,transiente,pack_name,fecha_ini,hora_ini,ruta_pesteo,prefijo_heads,mod2obs_in,sim_smp_file,pest_ins)

# Caudales de extracción

En esta sección se busca crear los archivos necesarios para incluir los caudales de extracción de aguas en el proceso de calibración. En este caso los pozos de bombeo son representados a través del paquete CLN y WEL.
En este caso se buscará calibrar el caudal de todos los pozos. 
Esta rutina sirve para cualquier paquete CLN que contenga exclusivamente pozos de bombeo (generalizada).
La instrucción que usa PEST se genera usando la función flow_wel2ins del módulo TargetsModule

In [4]:
ruta_bombeo='02 Bombeo\\'

#Archivo zone para usgbud2smp
zone_file = 'zone_cln.zone'

data_file = ruta_bombeo+'5315_Calibracion.cln'
data_file_delimiter = ' '
# Acá leemos el paquete CLN como un csv
df=readAnyCsv(data_file,data_file_delimiter)

# Acá obtenemos la cantidad de nodos y elementos CLN que contiene el paquete
number_of_elements = int(df[1][0])
number_of_nodes_cln = int(df[2][0])*-1
lines_from_internal = round(number_of_elements/10+0.5)

# Acá generamos el zone pozo por pozo
zone_cln = df[lines_from_internal+2:2+lines_from_internal+number_of_nodes][2]
# Finalmente generamos el archivo zone
zone_cln.to_csv(ruta_pesteo+zone_file,header=None, index=None, sep='\t')

# Ahora se lee el well info generado por GWV, para generar las respuestas que van a usgbud2smp
well_info_file='well-info.csv'
well_info=pd.read_csv(ruta_bombeo+well_info_file,',')

# Nombre del archivo que le dará respuestas al usgbud2smp
to_bud2smp_cln_file = 'to_bud2smp_cln.in'
# Archivo con caudales simulados
cln_sim_file = 'cln_sim.smp'

# Generamos las respuestas que van al usgbud2smp
answers_cln=[pack_name+'.cbcln',
                sp_totales,
                'gwf',
                fecha_ini,
                hora_ini,
                'd',
                number_of_nodes_cln,
                zone_file,
                'l',
                '1']
with open(ruta_pesteo+to_bud2smp_cln_file, 'w') as f:
    for item in answers_cln:
        f.write("%s\n" % item)
    for item in well_info['Name']:
        f.write("%s\n" % item)
    f.write(cln_sim_file)
    f.write('\n1')
    f.write('\nf')
    f.write('\nnul')

# Creamos la instrucción de PEST usando el TargetsModule
well_export_file='well_export.csv'
cln_ins_file = 'cln_sim.ins'
prefijo_wells='w'

wellcln2ins(ruta_bombeo,well_export_file,cln_ins_file,ruta_pesteo,prefijo_wells)

# Vertiente Jachucoposa

En esta sección se incorporarán a la interfaz de PEST los flujos observados de la vertiente Jachucoposa. Se construirán todos los elementos que necesita PEST para utilizar estos flujos como targets. Los datos de la vertiente fueron entregados en formato csv.

In [11]:
ruta_vertiente = '03 Vertiente\\'

identificador = 'drains'
transiente = pd.read_csv(ruta_vertiente+'5315_CaudalNaturalMedidoJachucoposa.csv',',')
prefijo_flux='j'
zone_file= 'drains.zone'
zones= ['jachu']
to_bud2smp_file='to_bud2smp_drn.in'
sim_smp_file='drn_sim.smp'
pest_ins='drn_sim.ins'

flux2all(identificador,transiente,pack_name,fecha_ini,hora_ini,ruta_pesteo,prefijo_flux,
             sp_totales,number_of_nodes,zone_file,zones,to_bud2smp_file,sim_smp_file,pest_ins)

# Evaporación

En esta sección se incorporará a la interfaz de PEST el flujo de evaporación. Se utilizará la misma función flux2all que en la sección anterior. 

In [13]:
ruta_evaporacion = '04 Evaporacion\\'

identificador = 'et'
transiente = pd.read_csv(ruta_evaporacion+'5315_target_evaporacion.csv',',')
prefijo_flux='e'
zone_file= 'ets.zone'
zones= ['ets']
to_bud2smp_file='to_bud2smp_ets.in'
sim_smp_file='ets_sim.smp'
pest_ins='ets_sim.ins'

flux2all(identificador,transiente,pack_name,fecha_ini,hora_ini,ruta_pesteo,prefijo_flux,
             sp_totales,number_of_nodes,zone_file,zones,to_bud2smp_file,sim_smp_file,pest_ins)

# Flujos pasantes

En esta sección se incorporarán los flujos pasantes a la interfaz de PEST.

In [3]:
# Primero creamos el input de zonbud
csv_file = 'zb_flujos_pasantes'
zone_file = 'flujos_pasantes.zone'
to_zb_file = 'to_zb_flujos.in'

inputzb(csv_file,pack_name,zone_file,ruta_pesteo,to_zb_file)

# Luego creamos el obs2obs


# Creación del PST

A continuación se construirá el pst usando PyEmu

Búsqueda de archivos TPL

In [5]:
tpl_files = [os.path.join(ruta_pesteo,f) for f in os.listdir(ruta_pesteo) if f.endswith(".tpl")]
input_files = [f.replace(".tpl",'') for f in tpl_files]
tpl_files

[]

Búsqueda de archivos INS

In [6]:
ins_files = [os.path.join(ruta_pesteo,f) for f in os.listdir(ruta_pesteo) if f.endswith(".ins")]
output_files = [f.replace(".ins",'.smp') for f in ins_files]
ins_files
output_files

['50 Pesteo\\cln_sim.smp', '50 Pesteo\\niveles_sim.smp']

In [7]:
pst = pyemu.Pst.from_io_files(tpl_files,input_files,ins_files,output_files)

C:\ProgramData\Anaconda3\lib\site-packages\pyemu\pst\pst_utils.py:615: UserWarning: error processing instruction file 50 Pesteo\cln_sim.ins, trying inschek: InstructionFile error processing output file on line number 122: unrecognized instruction '[w_ppc-02_1]38:50' on ins file line 2
  warnings.warn("error processing instruction file {0}, trying inschek: {1}".format(ins_file,str(e)))
C:\ProgramData\Anaconda3\lib\site-packages\pyemu\pst\pst_utils.py:615: UserWarning: error processing instruction file 50 Pesteo\niveles_sim.ins, trying inschek: output file '50 Pesteo\niveles_sim.smp' not found
  warnings.warn("error processing instruction file {0}, trying inschek: {1}".format(ins_file,str(e)))


error using inschek for instruction file 50 Pesteo\cln_sim.ins:run() returned non-zero: 1
observations in this instruction file will havegeneric values.
error using inschek for instruction file 50 Pesteo\niveles_sim.ins:run() returned non-zero: 1
observations in this instruction file will havegeneric values.


In [8]:
pst.write(ruta_pesteo+"5315_CAL.pst")

noptmax:30, npar_adj:0, nnz_obs:41999
